In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot, plot
import colormaps as cmaps
import matplotlib.colors as mc
import matplotlib.pyplot as plt


# define paths for data and output
paths = {"base": "...yourocalpath//FRESH//",  
         "data": "data//", 
         "fig": "figS1//",
         "savedir": "temp//",
        }


In [ ]:
# set colormap name
colormap_name = "tofino"
# get color map
cmap = getattr(cmaps, colormap_name)


In [ ]:
# loads pandas dataframe
df = pd.read_csv(paths["base"]+paths["data"]+"FreshData.csv", encoding='utf-16', delimiter='\t')

# unifies NaNs: Replace all "NaN" values in df with "nan"
df = df.replace(np.nan, 'nan', regex=True)

# specific for this analysis: replace all "NaN"s in df with "Unknown"
df = df.replace('nan', 'Unknown', regex=True)

# data for each study 
df_study1 = df[df['Study'] == 'Study 1'].copy()
df_study2 = df[df['Study'] == 'Study 2'].copy()

In [ ]:
# collect each teams responses in agreement / disagreement 
hyp127 = ['Hypothesis 1', 'Hypothesis 2', 'Hypothesis 7']
hyp3456 = [ 'Hypothesis 3', 'Hypothesis 4', 'Hypothesis 5', 'Hypothesis 6']
hypstudy2 = ['Study2 Hypothesis 1 Participant 1',
 'Study2 Hypothesis 1 Participant 2',
 'Study2 Hypothesis 1 Participant 3',
 'Study2 Hypothesis 1 Participant 4',
 'Study2 Hypothesis 1 Participant 5',
 'Study 2 Hypothesis 1 Participant 6',
 'Study 2 Hypothesis 1 Participant 7',
 'Study 2 Hypothesis 1 Participant 8',
 'Study 2 Hypothesis 1 Participant 9',
 'Study 2 Hypothesis 1 Participant 10',
 'Study 2 Hypothesis 2 Participant 1',
 'Study 2 Hypothesis 2 Participant 2',
 'Study 2 Hypothesis 2 Participant 3',
 'Study 2 Hypothesis 2 Participant 4',
 'Study 2 Hypothesis 2 Participant 5',
 'Study 2 Hypothesis 2 Participant 6',
 'Study 2 Hypothesis 2 Participant 7',
 'Study 2 Hypothesis 2 Participant 8',
 'Study 2 Hypothesis 2 Participant 9',
 'Study 2 Hypothesis 2 Participant 10',
 'Study 2 Hypothesis 3 Participant 1',
 'Study 2 Hypothesis 3 Participant 2',
 'Study 2 Hypothesis 3 Participant 3',
 'Study 2 Hypothesis 3 Participant 4',
 'Study 2 Hypothesis 3 Participant 5',
 'Study 2 Hypothesis 3 Participant 6',
 'Study 2 Hypothesis 3 Participant 7',
 'Study 2 Hypothesis 3 Participant 8',
 'Study 2 Hypothesis 3 Participant 9',
 'Study 2 Hypothesis 3 Participant 10',
 'Study 2 Hypothesis 4 Participant 1',
 'Study 2 Hypothesis 4 Participant 2',
 'Study 2 Hypothesis 4 Participant 3',
 'Study 2 Hypothesis 4 Participant 4',
 'Study 2 Hypothesis 4 Participant 5',
 'Study 2 Hypothesis 4 Participant 6',
 'Study 2 Hypothesis 4 Participant 7',
 'Study 2 Hypothesis 4 Participant 8',
 'Study 2 Hypothesis 4 Participant 9',
 'Study 2 Hypothesis 4 Participant 10']

# choose responses 
df_res127 = df_study1[hyp127].copy()
df_res3456 = df_study1[hyp3456].copy()
df_resstudy2 = df_study2[hypstudy2].copy()

# study 2 all hypotheses are expected true
df_resstudy2 = df_resstudy2.replace('Yes', 'In Agreement', regex=True) 
df_resstudy2 = df_resstudy2.replace('No', 'In Disagreement', regex=True) 

# study 1 all hypotheses are expected true, except hypothesis 7 is expected False 
expectedtrue = ['Hypothesis 1', 'Hypothesis 2']
for hyp in expectedtrue : 
    df_res127[hyp] =  df_res127[hyp].replace('Yes', 'In Agreement', regex=True) 
    df_res127[hyp] =  df_res127[hyp].replace('No', 'In Disagreement', regex=True) 
    
df_res127['Hypothesis 7'] =  df_res127['Hypothesis 7'].replace('No', 'In Agreement', regex=True) 
df_res127['Hypothesis 7'] =  df_res127['Hypothesis 7'].replace('Yes', 'In Disagreement', regex=True)

df_res3456 = df_res3456.replace('Yes', 'In Agreement', regex=True) 
df_res3456 = df_res3456.replace('No', 'In Disagreement', regex=True) 


In [ ]:
# prepare data for radar plot 1 

# selection of columns to keep for plot 1 
columns_sig = ['Study', 'Quality/Pruning: Method','Motion Artifact Method','Filtering' ]
df_siganalysis = df[columns_sig].copy()

# rename entries in columns indicated in column_none that contain an "Unknown" to the corresponding entry in none_txt
column_none = ['Quality/Pruning: Method', 'Motion Artifact Method']
none_txt = ['No Pruning', 'No Artifact Removal']
for i in range(len(column_none)):
    df_siganalysis[column_none[i]] = df_siganalysis[column_none[i]].replace('Unknown', none_txt[i], regex=True)

# Pruning
# rename "Other" entries in column 'Quality/Pruning: Method' to 'Other Pruning Method'
df_siganalysis['Quality/Pruning: Method'] = df_siganalysis['Quality/Pruning: Method'].replace('Other', 'Other Pruning Method', regex=True)
# In "Quality/Pruning: Method" rename "Manual Selection" and "Visual Inspection of Time Domain" to 'Manual Pruning
df_siganalysis['Quality/Pruning: Method'] = df_siganalysis['Quality/Pruning: Method'].replace('Visual Inspection of Time Domain', 'Manual Selection', regex=True)
df_siganalysis['Quality/Pruning: Method'] = df_siganalysis['Quality/Pruning: Method'].replace('Manual Selection', 'Manual Pruning', regex=True)
# identify rows with combined metrics
for i in range(len(df_siganalysis['Quality/Pruning: Method'].values)):
    if ',' in df_siganalysis['Quality/Pruning: Method'][i]: 
        df_siganalysis['Quality/Pruning: Method'][i] = 'Combined\nMetrics'
for i in range(len(df_siganalysis['Quality/Pruning: Method'].values)):
    if 'SCI (Scalp-Coupling Index)' in df_siganalysis['Quality/Pruning: Method'][i]: 
        df_siganalysis['Quality/Pruning: Method'][i] = 'Single Metric'
df_siganalysis['Quality/Pruning: Method'] = df_siganalysis['Quality/Pruning: Method'].replace('SNR Threshold', 'Single Metric', regex=True)   
df_siganalysis['Quality/Pruning: Method'] = df_siganalysis['Quality/Pruning: Method'].replace('Other Pruning Method', 'Single Metric', regex=True)    

# Motion Artefacts 
# rename 'No Artifact Removal' entries in column 'Motion Artifact Method' to 'Rejection' to later group in 'Trial Rejection/No Removal'
df_siganalysis['Motion Artifact Method'] = df_siganalysis['Motion Artifact Method'].replace('No Artifact Removal', 'Rejection', regex=True)
# rename "Rejection" entries in column 'Motion Artifact Method' to 'Trial Rejection/No Removal'
df_siganalysis['Motion Artifact Method'] = df_siganalysis['Motion Artifact Method'].replace('Rejection', 'Trial Rejection/\nNo Removal', regex=True)
# identify rows with combined metrics
for i in range(len(df_siganalysis['Motion Artifact Method'].values)):
    if ',' in df_siganalysis['Motion Artifact Method'][i]: 
        df_siganalysis['Motion Artifact Method'][i] = 'Combined Methods'
# identify rows with single method 
single_method =  {'Hybrid spline wavelet', 'Targeted PCA', 'Spline SG', 'Wavelet Filter',  'CBSI', 'Other Removal Method',  'TDDR', 'Other'}
for i in range(len(df_siganalysis['Motion Artifact Method'].values)):
    if  df_siganalysis['Motion Artifact Method'][i] in single_method:
        df_siganalysis['Motion Artifact Method'][i] = 'Single Method'   

# Filtering
# rename "HighPass,LowPass" and "LowPass,HighPass" entries in "Filtering" to "BandPass"
df_siganalysis['Filtering'] = df_siganalysis['Filtering'].replace('HighPass,LowPass', 'BandPass', regex=True)
df_siganalysis['Filtering'] = df_siganalysis['Filtering'].replace('LowPass,HighPass', 'BandPass', regex=True)
df_siganalysis['Filtering'] = df_siganalysis['Filtering'].replace('Unknown', 'No Filter', regex=True)

# choose data for study 1 and 2 
radar1_study1 = df_siganalysis[df_siganalysis['Study'] == 'Study 1']
radar1_study2 = df_siganalysis[df_siganalysis['Study'] == 'Study 2']


In [ ]:
# compute for each category percentage of in agreement / disagreement

# store data in these csv 
header = ['Options', 'In Agreement', 'In Disagreement']
data = [['Single Metric', 0, 0], ['Manual Pruning', 0, 0], ['No Pruning', 0, 0], ['Combined\nMetrics', 0, 0], ['Combined Methods', 0, 0], ['Single Method', 0, 0], ['Trial Rejection/\nNo Removal', 0, 0], ['No Filter', 0, 0], ['HighPass', 0, 0], ['BandPass', 0, 0], ['LowPass', 0, 0]]
study2pre = pd.DataFrame(data, columns=header)
study1h3456pre = pd.DataFrame(data, columns=header)
study1h127pre = pd.DataFrame(data, columns=header)

for col in ['Quality/Pruning: Method','Motion Artifact Method','Filtering' ]: 
    # column for study 1 
    column1 = radar1_study1[col]
    column1 = column1[ column1 != 'Not investigated']
    col_counts1 =  pd.Series(column1).value_counts(normalize= True)

    # column for study 2 
    column2 = radar1_study2[col]
    column2 = column2[ column2 != 'Not investigated']
    col_counts2 =  pd.Series(column2).value_counts(normalize= True)

    allmethods = set(column1)
    for method in allmethods: 
        scale1 = col_counts1[method]
        index1 = np.array([radar1_study1[col] == method])[0]
        answers127 = np.array(df_res127.iloc[index1])  
        yes_127 = np.sum(answers127 == 'In Agreement')
        no_127 = np.sum(answers127 == 'In Disagreement')
        study1h127pre.loc[study1h127pre['Options'] == method, 'In Agreement'] = (yes_127/(yes_127+ no_127)) * scale1 
        study1h127pre.loc[study1h127pre['Options'] == method, 'In Disagreement'] = (no_127/(yes_127+ no_127)) * scale1 
        answers3456 = np.array(df_res3456.iloc[index1])
        yes_3456 = np.sum(answers3456 == 'In Agreement')
        no_3456 = np.sum(answers3456 == 'In Disagreement')
        study1h3456pre.loc[study1h3456pre['Options'] == method, 'In Agreement'] = (yes_3456/(yes_3456+ no_3456)) * scale1 
        study1h3456pre.loc[study1h3456pre['Options'] == method, 'In Disagreement'] = (no_3456/(yes_3456+ no_3456)) * scale1 
        
    allmethods = set(column2)
    for method in allmethods:
        scale2 = col_counts2[method]
        index2 = np.array([radar1_study2[col] == method])[0]
        answersstudy2 = np.array(df_resstudy2.iloc[index2])
        yes_study2 = np.sum(answersstudy2 == 'In Agreement')
        no_study2 = np.sum(answersstudy2 == 'In Disagreement')
        study2pre.loc[study2pre['Options'] == method, 'In Agreement'] = (yes_study2/(yes_study2+ no_study2)) * scale2 
        study2pre.loc[study2pre['Options'] == method, 'In Disagreement'] = (no_study2/(yes_study2+ no_study2))  * scale2 


In [ ]:
# prepare data for radar plot 2 

# selection of columns to keep for plot 2 
columns_sig = ['Study',  'Non-GLM: Method', 'GLM: Method', 'GLM: HRF Regressor', 'GLM: Other Regressors']
radar2 = df[columns_sig].copy()
for i in range(len(radar2['GLM: Other Regressors'].values)):
    if ',' in radar2['GLM: Other Regressors'][i]: 
        radar2['GLM: Other Regressors'][i] = 'Multiple Regressors'

radar2['GLM: Other Regressors'] = radar2['GLM: Other Regressors'].replace('Unknown', 'Unknown Regressor', regex=True)
radar2['GLM: Method'] = radar2['GLM: Method'].replace('OLS   ', 'OLS', regex=True)
radar2['GLM: HRF Regressor'] = radar2['GLM: HRF Regressor'].replace('Consecutive Gaussian', 'Consecutive\nGaussian', regex=True)
rows = [['Block Averaging'], ['AR-IRLS', 'OLS'], ['Consecutive\nGaussian', 'FIR', 'Canonical', 'SPM', 'Glover','Gamma' ], 
        ['Multiple Regressors', 'Short Channels', 'PCA of the Short Channels', 'Unknown Regressor']]

radar2_study1= radar2[radar2['Study'] == 'Study 1']
radar2_study2= radar2[radar2['Study'] == 'Study 2']

# store data in these csv 
header = ['Options', 'In Agreement', 'In Disagreement']
data = [[ opt, 0, 0] for i in range(len(rows)) for opt in rows[i] ]
study2hrf =  pd.DataFrame(data, columns=header)
study1h3456hrf = pd.DataFrame(data, columns=header)
study1h127hrf = pd.DataFrame(data, columns=header)


# compute ratios for radar plot 2 
for col in columns_sig[1:]: 
    
    column1 = radar2_study1[col]
    column1 = column1[ column1 != 'Not investigated']
    col_counts1 =  pd.Series(column1).value_counts(normalize= True)
    
    column2 = radar2_study2[col]
    column2 = column2[ column2 != 'Not investigated']
    col_counts2 =  pd.Series(column2).value_counts(normalize= True)
    
    allmethods = set(column1)
    for method in allmethods: 
        scale1 = col_counts1[method]
        index1 = np.array([radar2_study1[col] == method])[0]
        answers127 = np.array(df_res127.iloc[index1])
        yes_127 = np.sum(answers127 == 'In Agreement')
        no_127 = np.sum(answers127 == 'In Disagreement')
        study1h127hrf.loc[study1h127hrf['Options'] == method, 'In Agreement'] = (yes_127/(yes_127+ no_127)) * scale1 
        study1h127hrf.loc[study1h127hrf['Options'] == method, 'In Disagreement'] = (no_127/(yes_127+ no_127)) * scale1 
        answers346 = np.array(df_res3456.iloc[index1])
        yes_346 = np.sum(answers346 == 'In Agreement')
        no_346 = np.sum(answers346 == 'In Disagreement')
        study1h3456hrf.loc[study1h3456hrf['Options'] == method, 'In Agreement'] = (yes_3456/(yes_3456+ no_3456)) * scale1 
        study1h3456hrf.loc[study1h3456hrf['Options'] == method, 'In Disagreement'] = (no_3456/(yes_3456+ no_3456)) * scale1 
        
    allmethods = set(column2)
    for method in allmethods:
        scale2 = col_counts2[method]
        index2 = np.array([radar2_study2[col] == method])[0]
        answersstudy2 = np.array(df_resstudy2.iloc[index2])
        yes_study2 = np.sum(answersstudy2 == 'In Agreement')
        no_study2 = np.sum(answersstudy2 == 'In Disagreement')
        study2hrf.loc[study2hrf['Options'] == method, 'In Agreement'] = (yes_study2/(yes_study2+ no_study2)) * scale2 
        study2hrf.loc[study2hrf['Options'] == method, 'In Disagreement'] = (no_study2/(yes_study2+ no_study2))  * scale2 



In [ ]:
# prepare data for radar plot 3 
columns_sig = [ 'Study', 'Stat Analysis: On Signal Type', 'Stat Analysis: Signal Space','Stat Analysis: Metric']
radar3 = df[columns_sig].copy()

# signal type 
for i in range(len(radar3[ 'Stat Analysis: On Signal Type'].values)):
    if ',' in radar3[ 'Stat Analysis: On Signal Type'][i]: 
        radar3[ 'Stat Analysis: On Signal Type'][i]= 'Other'
radar3[ 'Stat Analysis: On Signal Type'] = radar3[ 'Stat Analysis: On Signal Type'].replace('Other', 'Other/Multiple\nSignal Types', regex=True)

# signal space 
for i in range(len(radar3['Stat Analysis: Signal Space'].values)):
    if ',' in radar3['Stat Analysis: Signal Space'][i]: 
        radar3['Stat Analysis: Signal Space'][i]= 'Other'      
#radar3['Stat Analysis: Signal Space'] = radar3[ 'Stat Analysis: Signal Space'].replace('Unknown', 'Other', regex=True)
radar3['Stat Analysis: Signal Space'] = radar3[ 'Stat Analysis: Signal Space'].replace('Other', 'Other/Multiple\nSignal Spaces', regex=True)

# metric 
for i in range(len(radar3['Stat Analysis: Metric'])): 
    if radar3['Stat Analysis: Metric'][i] not in {'signal amplitude (windowed)', 'Not investigated', 'beta values provided by GLM'}: 
        radar3['Stat Analysis: Metric'][i] = 'Other Metric'
        
radar3['Stat Analysis: Metric'] = radar3[ 'Stat Analysis: Metric'].replace('beta values provided by GLM', 'GLM Beta Values', regex=True)
radar3['Stat Analysis: Metric'] = radar3['Stat Analysis: Metric'].str.strip().replace('signal amplitude (windowed)', 'Signal Amp. (windowed)', regex=False)


rows = [[ 'HbO only', 'HbO and HbR', 'Other/Multiple\nSignal Types'], [ 'Channel', 'Other/Multiple\nSignal Spaces', 'ROI'], ['GLM Beta Values', 'Signal Amp. (windowed)', 'Other Metric']]
# store data in these csv 
header = ['Options', 'In Agreement', 'In Disagreement']
data = [[ opt, 0, 0] for i in range(len(rows)) for opt in rows[i] ]
study2stan =  pd.DataFrame(data, columns=header)
study1h3456stan = pd.DataFrame(data, columns=header)
study1h127stan = pd.DataFrame(data, columns=header)

radar3_study1= radar3[radar3['Study'] == 'Study 1']
radar3_study2= radar3[radar3['Study'] == 'Study 2']

# compute ratios for radar plot 3 
for col in columns_sig[1:]: 
    
    column1 = radar3_study1[col]
    column1 = column1[ column1 != 'Not investigated']
    col_counts1 =  pd.Series(column1).value_counts(normalize= True)
    
    column2 = radar3_study2[col]
    column2 = column2[ column2 != 'Not investigated']
    col_counts2 =  pd.Series(column2).value_counts(normalize= True)
    
    allmethods = set(column1)
    for method in allmethods: 
        scale1 = col_counts1[method]
        index1 = np.array([radar3_study1[col] == method])[0]
        answers127 = np.array(df_res127.iloc[index1])
        yes_127 = np.sum(answers127 == 'In Agreement')
        no_127 = np.sum(answers127 == 'In Disagreement')
        study1h127stan.loc[study1h127stan['Options'] == method, 'In Agreement'] = (yes_127/(yes_127+ no_127)) * scale1 
        study1h127stan.loc[study1h127stan['Options'] == method, 'In Disagreement'] = (no_127/(yes_127+ no_127)) * scale1 
        answers3456 = np.array(df_res3456.iloc[index1])
        yes_3456 = np.sum(answers3456 == 'In Agreement')
        no_3456 = np.sum(answers3456 == 'In Disagreement')
        study1h3456stan.loc[study1h3456stan['Options'] == method, 'In Agreement'] = (yes_3456/(yes_3456+ no_3456)) * scale1 
        study1h3456stan.loc[study1h3456stan['Options'] == method, 'In Disagreement'] = (no_3456/(yes_3456+ no_3456)) * scale1 
        
    allmethods = set(column2)
    for method in allmethods:
        scale2 = col_counts2[method]
        index2 = np.array([radar3_study2[col] == method])[0]
        answersstudy2 = np.array(df_resstudy2.iloc[index2])
        yes_study2 = np.sum(answersstudy2 == 'In Agreement')
        no_study2 = np.sum(answersstudy2 == 'In Disagreement')
        study2stan.loc[study2stan['Options'] == method, 'In Agreement'] = (yes_study2/(yes_study2+ no_study2)) * scale2 
        study2stan.loc[study2stan['Options'] == method, 'In Disagreement'] = (no_study2/(yes_study2+ no_study2))  * scale2 



In [ ]:
# prepare data for radar plot 4 
columns_sig = ['Study', 'Stat Analysis: Method', 'Multiple Comparisons Correction', 'Stat Analysis: Threshold (p< X)']
radar4 = df[columns_sig].copy()

# Stat. Method
for i in range(len(radar4[ 'Stat Analysis: Method'].values)):
    if ',' in radar4[ 'Stat Analysis: Method'][i]: 
        radar4[ 'Stat Analysis: Method'][i]= 'Multiple Methods'   
for i in range(len(radar4[ 'Stat Analysis: Method'].values)):
    if radar4[ 'Stat Analysis: Method'][i] in ['One-Sample t-Test', 'Paired t-Test', 'Two-tailed t-test' ]:
        radar4[ 'Stat Analysis: Method'][i]= 'T-Tests'  
    if radar4[ 'Stat Analysis: Method'][i] in ['Robust Mixed Effects Model', 'Bootstrap Estimation', 'Hotelling T2 test', 'Wilcoxon signed-rank test', 'Repeated Measures ANOVA', 'Linear Mixed Effects', 'Fixed Mixed Effects Model', 'Random Effects', 'Mixed Effects Model', 'Permutation Test']:
        radar4[ 'Stat Analysis: Method'][i]= 'Mixed Effects Models'
    if radar4[ 'Stat Analysis: Method'][i] in ['Other', 'Unknown' ]:
        radar4[ 'Stat Analysis: Method'][i]= 'Other/Unknown \nStat. Method'

# Multiple Comparisons Correction
radar4[ 'Multiple Comparisons Correction'] = radar4['Multiple Comparisons Correction'].replace('Benjamini-Hochberg', 'Correction', regex=True)
radar4[ 'Multiple Comparisons Correction'] = radar4['Multiple Comparisons Correction'].replace('Bonferroni', 'Correction', regex=True)
radar4[ 'Multiple Comparisons Correction'] = radar4['Multiple Comparisons Correction'].replace('Benjamini-Yekutieli', 'Correction', regex=True)
radar4[ 'Multiple Comparisons Correction'] = radar4['Multiple Comparisons Correction'].replace('Unknown', 'No Correction', regex=True)

# p-val 
radar4[ 'Stat Analysis: Threshold (p< X)'] = radar4['Stat Analysis: Threshold (p< X)'].replace( '0.05' , 'alpha = 0.05', regex=True)
#radar4[ 'Stat Analysis: Threshold (p< X)'] = radar4['Stat Analysis: Threshold (p< X)'].replace( 'Unknown' , 'Unknown p-', regex=True)
for p in ['0.001', '0.025', '0.01']: 
    radar4[ 'Stat Analysis: Threshold (p< X)'] = radar4['Stat Analysis: Threshold (p< X)'].replace( p , 'alpha < 0.05', regex=True)

# store data in these csv
rows = [[ 'Multiple Methods','Other/Unknown \nStat. Method', 'T-Tests', 'Mixed Effects Models'], [ 'Correction', 'No Correction'], ['alpha < 0.05', 'alpha = 0.05']] 
header = ['Options', 'In Agreement', 'In Disagreement']
data = [[ opt, 0, 0] for i in range(len(rows)) for opt in rows[i] ]
study2test =  pd.DataFrame(data, columns=header)
study1h3456test = pd.DataFrame(data, columns=header)
study1h127test = pd.DataFrame(data, columns=header)

radar4_study1= radar4[radar4['Study'] == 'Study 1']
radar4_study2= radar4[radar4['Study'] == 'Study 2']

# compute ratios for radar plot 4
for col in columns_sig[1:]: 
    
    column1 = radar4_study1[col]
    column1 = column1[ column1 != 'Not investigated']
    col_counts1 =  pd.Series(column1).value_counts(normalize= True)
    
    column2 = radar4_study2[col]
    column2 = column2[ column2 != 'Not investigated']
    col_counts2 =  pd.Series(column2).value_counts(normalize= True)
    
    allmethods = set(column1)
    for method in allmethods: 
        scale1 = col_counts1[method]
        
        index1 = np.array([radar4_study1[col] == method])[0]
        answers127 = np.array(df_res127.iloc[index1])
        yes_127 = np.sum(answers127 == 'In Agreement')
        no_127 = np.sum(answers127 == 'In Disagreement')
        study1h127test.loc[study1h127test['Options'] == method, 'In Agreement'] = (yes_127/(yes_127+ no_127)) * scale1 
        study1h127test.loc[study1h127test['Options'] == method, 'In Disagreement'] = (no_127/(yes_127+ no_127)) * scale1 
        
        answers3456 = np.array(df_res3456.iloc[index1])
        yes_3456 = np.sum(answers3456 == 'In Agreement')
        no_3456 = np.sum(answers3456 == 'In Disagreement')
        study1h3456test.loc[study1h3456test['Options'] == method, 'In Agreement'] = (yes_3456/(yes_3456+ no_3456)) * scale1 
        study1h3456test.loc[study1h3456test['Options'] == method, 'In Disagreement'] = (no_3456/(yes_3456+ no_3456)) * scale1 
        
    allmethods = set(column2)
    for method in allmethods:
        scale2 = col_counts2[method]
        index2 = np.array([radar4_study2[col] == method])[0]
        answersstudy2 = np.array(df_resstudy2.iloc[index2])
        yes_study2 = np.sum(answersstudy2 == 'In Agreement')
        no_study2 = np.sum(answersstudy2 == 'In Disagreement')
        study2test.loc[study2test['Options'] == method, 'In Agreement'] = (yes_study2/(yes_study2+ no_study2)) * scale2 
        study2test.loc[study2test['Options'] == method, 'In Disagreement'] = (no_study2/(yes_study2+ no_study2))  * scale2 

In [ ]:
fig, axs = plt.subplots( figsize= (8.27, 11.69), nrows=4, ncols=3, subplot_kw={'projection': 'polar'})

data = [[study1h127pre, study1h3456pre,  study2pre], [study1h127hrf,  study1h3456hrf, study2hrf  ], [study1h127stan,study1h3456stan, study2stan ] , [study1h127test,study1h3456test, study2test ]]

# choose colors 
blue = cmap(0.2)
green  = cmap(0.7)
backgroundcolors = [ cmap(0.1), cmap(0.3), cmap(0.8)]
for i in range(4):
    for j in range(3): 
        ax = axs[i,j]
        # pick data for radar plot 
        D = np.array(data[i][j].values)
    
        methods = list(D[:, 0])
        methods = [*methods, methods[0]]
        n_points = len(methods)
        # compute angles of labels 
        label_loc = np.linspace(start=0, stop=2 * np.pi, num=len(methods)) 
        
        group_1 = D[:, 1] * 100 
        group_1 = [*group_1, group_1[0]] 
        group_2 = D[:, 2] * 100 
        group_2 = [*group_2, group_2[0]]
        
        # ensure plot with smaller amplitudes is on top 
        if (i,j) in {(0,1), (0,2), (1,1), (2,1), (2,2), (3,1)}:
            ax.plot(label_loc, group_2, color= blue, label='In Disagreement')
            ax.plot(label_loc, group_1, color= green , label='In Agreement')
        
        else: 
            ax.plot(label_loc, group_1, color= green , label='In Agreement')
            ax.plot(label_loc, group_2, color= blue, label='In Disagreement')

        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        max_r = np.maximum(np.max(group_1), np.max(group_2))
        
        # Change the color of the circular gridlines.
        ax.grid(color='#AAAAAA') 
        # Change the color of the outermost gridline
        ax.spines['polar'].set_color('#eaeaea') 
        # Change the background color inside the circle itself
        ax.set_facecolor('#FAFAFA')
        
        
        # Draw axis lines for each angle and label.
        methods[-1] = ''

        label_loc2 = np.degrees(label_loc).copy()

        # adjust labels and color pie parts 
        # add colored labels on the left side 
        if i == 0: 
            label_loc2[2] = label_loc2[2] + 2
            label_loc2[3] = label_loc2[3] + 4
            label_loc2[8] = label_loc2[8] + 2
            label_loc2[9] = label_loc2[9] + 2
            
            halfdeg = label_loc[1]/2 
            begin  = [label_loc[0], label_loc[4] , label_loc[7]]
            end = [ label_loc[3],  label_loc[6], label_loc[10]]
            heading = ['Pruning', 'Motion Artefacts', 'Filtering']
            for k in range(len(begin)): 
                t = np.linspace(begin[k] -halfdeg , end[k] + halfdeg, 100)
                ax.fill_between(t, 0, max_r, color = backgroundcolors[k], alpha = 0.3)
                if j == 0: 
                    fig.text(0.01, 0.87 -0.01* k, heading[k], fontsize=6, color = backgroundcolors[k] )

            
        if i == 1: 
            label_loc2[2] = label_loc2[2] + 2
            label_loc2[3] = label_loc2[3] + 4
            label_loc2[4] = label_loc2[4] + 2
            label_loc2[9] = label_loc2[9] + 2
            label_loc2[10] = label_loc2[10] + 2
            label_loc2[11] = label_loc2[11] + 2

            halfdeg = label_loc[1]/2 
            begin  = [label_loc[0], label_loc[3] , label_loc[9]]
            end = [ label_loc[2],  label_loc[8], label_loc[12]]
            heading = ['HRF Estimation', 'HRF Regressors', 'Other Regressors']
            for k in range(len(begin)): 
                t = np.linspace(begin[k] -halfdeg , end[k] + halfdeg, 100)
                ax.fill_between(t, 0, max_r, color = backgroundcolors[k], alpha = 0.3)
                if j == 0: 
                    fig.text(0.01, 0.67 -0.01* k, heading[k], fontsize=6, color = backgroundcolors[k] )

            
        if i == 2: 
            label_loc2[2] = label_loc2[2] + 4
            label_loc2[3] = label_loc2[3] + 2
            label_loc2[6] = label_loc2[6] + 2
            label_loc2[7] = label_loc2[7] + 4
            
            halfdeg = label_loc[1]/2 
            begin  = [label_loc[0], label_loc[3] , label_loc[6]]
            end = [ label_loc[2],  label_loc[5], label_loc[8]]
            heading = ['Signal Type', 'Signal Space', 'Metric']
            for k in range(len(begin)): 
                t = np.linspace(begin[k] -halfdeg , end[k] + halfdeg, 100)
                ax.fill_between(t, 0, max_r, color = backgroundcolors[k], alpha = 0.3)
                if j == 0: 
                    fig.text(0.01, 0.49 -0.01* k, heading[k], fontsize=6, color = backgroundcolors[k] )
            
        if i == 3: 
            label_loc2[2] = label_loc2[2] + 2
            label_loc2[3] = label_loc2[3] + 2
            label_loc2[6] = label_loc2[6] + 2
            label_loc2[7] = label_loc2[7] + 2
            halfdeg = label_loc[1]/2 
            begin  = [label_loc[0], label_loc[4] , label_loc[6]]
            end = [ label_loc[3],  label_loc[5], label_loc[7]]
            heading = ['Statistical Method', 'Multiple Comparisons Correction', 'p-Value']
            for k in range(len(begin)): 
                t = np.linspace(begin[k] -halfdeg , end[k] + halfdeg, 100)
                ax.fill_between(t, 0, max_r, color = backgroundcolors[k], alpha = 0.3)
                if j == 0: 
                    fig.text(0.01, 0.27 -0.01* k, heading[k], fontsize=6, color = backgroundcolors[k] )
        

        # Set the positions of the labels
        ax.set_thetagrids(np.degrees(label_loc), '')
        label_angles = np.deg2rad(label_loc2)
        label_distances = np.ones_like(label_angles) * (max_r * (1.1)) 
        for angle, distance, method in zip(label_angles, label_distances, methods):
            ax.text(angle, distance, method, fontsize=5, ha='center', va='center')
        for label, angle in zip(ax.get_xticklabels(), label_loc):
            if np.pi/2 < angle: 
                label.set_horizontalalignment('right')
            if 0 < angle < np.pi/2: 
                label.set_horizontalalignment('left')
            else: 
                label.set_horizontalalignment('center')
                
                
        # make y axis label smaller 
        ticks = ax.get_yticks()
        ax.tick_params(axis='y', label1On=False)
        angle = [50, 100, 100, 50 ]
        for k in range(len(ticks)-1): 
            ax.text(np.deg2rad(angle[i]), ticks[k], str(int(ticks[k])),fontsize=6,  ha='center', va='center')

            
# add legend 
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles[:2], labels[:2], loc=(0.85,0.935), fontsize = 6)
# set titles 
axs[0,0].set_title('Study 1 Hypotheses 1, 2 & 7\n (strong literature support)', fontsize=8, pad = 30 )
axs[0,1].set_title('Study 1 Hypotheses 3, 4, 5 & 6\n (weak literature support)', fontsize=8, pad = 30 )
axs[0,2].set_title('Study 2 all Hypotheses\n (strong & weak literature support)', fontsize=8, pad =30 )
axs[0,0].set_ylabel('Preprocessing', labelpad = 50 )
axs[1,0].set_ylabel('HRF Extraction', labelpad = 50 )
axs[2,0].set_ylabel('Signals for Statistical\nAnalysis', labelpad = 40 )
axs[3,0].set_ylabel('Statistical Tests', labelpad = 50 )
fig.suptitle('Distribution of choices in pipelines for signal processing and statistical analysis\nbased on their (dis-)agreement with expected hypotheses outcomes', fontsize=11)



#plt.savefig(paths["base"]+paths["data"]+paths["fig"]+paths["savedir"]+'radarchart.pdf', format = 'pdf')
